In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import os

In [ ]:
# Load the data from the directory
# The data is in the following format:
# data
# ├── class1
# │   ├── image1.jpg
# │   ├── image2.jpg
# │   └── ...
# ├── class2
# │   ├── image1.jpg
# │   └── ...
# └── ...
data = tf.keras.utils.image_dataset_from_directory('data')
data_iterator = data.as_numpy_iterator()

In [ ]:
# Remap the scaled data
data = data.map(lambda x, y: (x/255, y))

In [ ]:
# Get the iterator for the scaled data
scaled_iterator = data.as_numpy_iterator()

In [ ]:
# Set the train, validation, and test sizes
train_size = int(len(data)*.6) + 2
val_size = int(len(data)*.3) - 1
test_size = int(len(data)*.1) + 1

In [ ]:
# Split the data into train, validation, and test sets
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
# Create a simple CNN sequential model, with 3 convolutional layers, 3 max pooling layers, and 2 dense layers
model = Sequential()
model.add(Conv2D(16, (3, 3), 1, activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model with the Adam optimizer and binary crossentropy loss
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
# Create the tensorboard callback for all logs during training
logdir='logs'
tensorboard__callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# Train the model with the train and validation data
# epochs are the number of times the model will see the data and adjust the weights by training
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard__callback])

In [ ]:
# Show the accuracy and loss of the model
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Show the accuracy and validation accuracy of the model
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
from keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
# Set the metrics for the model
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
# Evaluate the model with the test data
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
import cv2 as cv

In [ ]:
# Load the image and resize it to 256x256
img = cv.imread('test.jpg')
resize = tf.image.resize(img, (256, 256))

In [ ]:
# Predict the class of the image and save it
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
# Print the class of the image, it is < 0.5 for class 1, and > 0.5 for class 2 because we have binary classification
if yhat < 0.5:
    print("Class 1")
else:
    print("Class 2")

In [ ]:
# Save the model
model.save('model_name.h5')

In [ ]:
from keras.models import load_model

In [ ]:
# Let's see how to make a prediction with a saved model
# Load the model
new_model = load_model('model_name.h5')

In [ ]:
# Predict the class of the image and save it
yhat_new = new_model.predict(np.expand_dims(resize/255, 0))

In [ ]:
# It is the same as before, but we are using the new model
if yhat_new < 0.5:
    print("Class 1")
else:
    print("Class 2")